In [29]:
!pip install implicit

  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\79609\anaconda3\python.exe' 'C:\Users\79609\anaconda3\lib\site-packages\pip\_vendor\pep517\_in_process.py' build_wheel 'C:\Users\79609\AppData\Local\Temp\tmph49ao05_'
       cwd: C:\Users\79609\AppData\Local\Temp\pip-install-1pak_twt\implicit
  Complete output (35 lines):
  Failed to find CUDA toolkit. Building without GPU acceleration.
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.8
  creating build\lib.win-amd64-3.8\implicit
  copying implicit\als.py -> build\lib.win-amd64-3.8\implicit
  copying implicit\approximate_als.py -> build\lib.win-amd64-3.8\implicit
  copying implicit\bpr.py -> build\lib.win-amd64-3.8\implicit
  copying implicit\nearest_neighbours.py -> build\lib.win-amd64-3.8\implicit
  copying implicit\utils.py -> build\lib.win-amd64-3.8\implicit
  copying implicit\__init__.py -> build\lib.win-amd64-3.8\implicit
  creating build\lib.win-amd64-


  Using cached implicit-0.4.8.tar.gz (1.1 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
Failed to build implicit


In [30]:
import pandas as pd

import numpy as np

from scipy.sparse import csr_matrix, coo_matrix

from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

ModuleNotFoundError: No module named 'implicit'

In [31]:
data = pd.read_csv('retail_train.csv')

In [32]:
data.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [33]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

#### 1 Weighted random recommender

In [34]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    items = np.array(items_probabilities['item_id'])
    probability = np.array(items_probabilities['probability'])
    
    recs = np.random.choice(items, size=n, replace=False, p=probability)
    
    return recs.tolist()

In [35]:
%%time

data = data_train
items_probabilities = data.groupby('item_id')['sales_value'].sum().reset_index()
items_probabilities['log'] = np.log1p(items_probabilities['sales_value'])
sum_ = items_probabilities.log.sum()
items_probabilities['probability'] = items_probabilities['log']/sum_.sum()
items_probabilities.probability.sum()

Wall time: 177 ms


1.0

In [36]:
weighted_random_recommendation(items_probabilities,5)

[1127227, 8090919, 7466787, 1088841, 13038706]

#### 2. Измерим качество по precision@5

In [37]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()

In [38]:
result.head()

,user_id,item_id
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."


In [39]:
def t(df):
    for j in range(df.shape[1]):
        print(type(df.iloc[0][j]))

In [40]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision

In [41]:
def List(result):
    for i in range(result.shape[0]):
        tt = 'actual'
        result[tt][i] = [int(x) for x in list(result[tt][i].replace('[','').replace(']','').split(', '))]
        tt = 'random_recommendation'
        result[tt][i] = [int(x) for x in list(result[tt][i].replace('[','').replace(']','').split(', '))]
        tt = 'popular_recommendation'
        result[tt][i] = [int(x) for x in list(result[tt][i].replace('[','').replace(']','').split(', '))]
        tt = 'itemitem'
        result[tt][i] = [int(x) for x in list(result[tt][i].replace('[','').replace(']','').split(', '))]
        tt = 'cosine'
        result[tt][i] = [int(x) for x in list(result[tt][i].replace('[','').replace(']','').split(', '))]
        tt = 'tfidf'
        result[tt][i] = [int(x) for x in list(result[tt][i].replace('[','').replace(']','').split(', '))]
        tt = 'own_purchases'
        result[tt][i] = [int(x) for x in list(result[tt][i].replace('[','').replace(']','').split(', '))]

In [42]:
t(result)

<class 'numpy.int64'>
<class 'numpy.ndarray'>


In [ ]:
List(result)